In [9]:
# Najpierw trzeba sprawdzić wyniki z pliku ImageResults -> jaki jest accuracy
# W każdej linii - pierwszy znak T lub F oznacza oczekiwany rezultat, a ostatnia wartość (authentic/spoof) - uzyskany
THRESHOLD = 0.50

In [10]:
file = "ImageResults.txt"

with open(file, "r") as f:
    lines = f.readlines()

# Initialize counters
correct_predictions = 0
total_predictions = 0

# Iterate through each line in the file
for line in lines:
    # Split the line into parts
    parts = line.strip().split(" ")

    expected_result = line[0]
    obtained_result = parts[-1].strip()  # Obtained result (authentic/spoof)

    # Check if the expected result matches the obtained result
    if expected_result == "T" and obtained_result == "authentic":
        correct_predictions += 1
    elif expected_result == "F" and obtained_result == "spoof":
        correct_predictions += 1

    total_predictions += 1

accuracy = correct_predictions / total_predictions * 100

print(f"Accuracy: {accuracy:.2f}%")
# TO JEST Z 55 threshold

Accuracy: 98.77%


In [13]:
# Wczytanie danych aby puścić już wytrenowany model
import cv2
import os

auth_path = "/Users/stukeleyak/Desktop/Studia/Doktorat/Projekt/FaceAuthenticityDetection/TestyInnychSystemów/PyTorchModelsTests/dane/CelebA/authentic"
spoof_path = "/Users/stukeleyak/Desktop/Studia/Doktorat/Projekt/FaceAuthenticityDetection/TestyInnychSystemów/PyTorchModelsTests/dane/CelebA/spoof"

auth_images = [os.path.join(auth_path, img) for img in os.listdir(auth_path)]
spoof_images = [os.path.join(spoof_path, img) for img in os.listdir(spoof_path)]

print("Loaded images:")
print(f"Authentic: {len(auth_images)}")
print(f"Spoof: {len(spoof_images)}")

Loaded images:
Authentic: 13543
Spoof: 9509


In [17]:
# Wyniki dla modelu ConvNeXt
import torch
from torchvision import transforms, models
from torch import nn, optim
from torch.utils.data import DataLoader
from PIL import Image

model = models.convnext_small(weights=None)
num_ftrs = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(num_ftrs, 2)  # Two output neurons

# Load the model weights
path = "/Users/stukeleyak/Desktop/Studia/Doktorat/Projekt/FaceAuthenticityDetection/CrossWalidacja/best_model_convnext.pth"
model.load_state_dict(torch.load(path))
model.eval()

# Transform the images
# Create a transform to resize images
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Create custom datasets
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label, image_path  # Return image, label, and file path


# Create custom datasets
test_dataset = OurDataset(auth_images + spoof_images, [0] * len(auth_images) + [1] * len(spoof_images), transform=transform)

# Create dataloaders
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model = model.to(device)

# Create a text file to store incorrect predictions
with open("incorrect_predictions_convnext.txt", "w") as f:
    f.write("name\tpredicted\tactual\n")
    f.write("(0 - authentic, 1 - spoof)\n")

correct_predictions = 0
total_predictions = 0

# Iterate through the test dataset
for images, labels, image_paths in test_loader:
    images = images.to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = model(images)

    # Get the argmax of each output
    _, predicted = torch.max(outputs, 1)

    # TODO: aktualnie predykcje sa tylko 0/1, moze trzeba zmienic na prawdopodobienstwa softmaxem

    total_predictions += labels.size(0)

    correct_predictions += (predicted == labels).sum().item()

    # Write all predictions to the file
    with open("incorrect_predictions_convnext.txt", "a") as f:
        for i in range(len(predicted)):
            f.write(f"{image_paths[i]}\t{predicted[i].item()}\t{labels[i].item()}\n")

print("Accuracy for ConvNeXt model:")
accuracy = correct_predictions / total_predictions * 100
print(f"Accuracy: {accuracy:.2f}%")

/var/folders/42/03wqtylj4nz_4lpvj_w6mc2h0000gn/T/ipykernel_19957/2402830029.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))


Accuracy for ConvNeXt model:
Accuracy: 94.52%


In [27]:
# Prepare data for the final NN
# [M1, M2, M3, M4, M5, Expected]
import pandas as pd
import numpy as np

# Create two maps: (name -> M5) and (name -> M1..M4)
# Then create rows like [M1, M2, ..., M5, expected]

# Create a dictionary to store the data
data_dict = {}
new_data_dict = {}

df = pd.DataFrame(columns=["M1", "M2", "M3", "M4", "M5", "Expected"])

# Read the file and populate the dictionary
with open("ImageResults.txt", "r") as f:
    lines = f.readlines()

    lines = lines[1:]  # Skip the first line

    for line in lines:
        parts = line.strip().split(" ")
        name = parts[0]
        m1 = float(parts[1])
        m2 = float(parts[2])
        m3 = float(parts[3])
        m4 = float(parts[4])

        new_data_dict[name] = [m1, m2, m3, m4]

# Read second file
with open("incorrect_predictions_convnext.txt", "r") as f:
    lines = f.readlines()

    lines = lines[2:]  # Skip the first two lines

    for line in lines:
        parts = line.strip().split("\t")
        name = parts[0]
        name = name.split("/")[-1]  # Get the file name only
        predicted = float(parts[1])
        expected = int(parts[2])

        if name in new_data_dict:
            m1m4 = new_data_dict[name]
            m1 = m1m4[0]
            m2 = m1m4[1]
            m3 = m1m4[2]
            m4 = m1m4[3]

            row = [m1, m2, m3, m4, predicted, expected]

            # Add row to df
            df = pd.concat([df, pd.DataFrame([row], columns=["M1", "M2", "M3", "M4", "M5", "Expected"])], ignore_index=True)

print("Dataframe length:")
print(len(df))

/var/folders/42/03wqtylj4nz_4lpvj_w6mc2h0000gn/T/ipykernel_19957/3310867508.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row], columns=["M1", "M2", "M3", "M4", "M5", "Expected"])], ignore_index=True)


Dataframe length:
22324


In [32]:
# Final NN
# Parameter to control the amount of inputs to the NN
N_INPUTS = 5

class ProbabilityNN(nn.Module):
    def __init__(self):
        super(ProbabilityNN, self).__init__()
        self.fc1 = nn.Linear(N_INPUTS, 10)
        self.fc2 = nn.Linear(10, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [36]:
def train_probability_nn(df):
    inputs = df[["M1", "M2", "M3", "M4", "M5"]]
    outputs = df[["Expected"]]
    outputs = outputs.astype(np.float32)

    model = ProbabilityNN()

    dataset = torch.utils.data.TensorDataset(torch.tensor(inputs.values, dtype=torch.float32), torch.tensor(outputs.values, dtype=torch.float32))
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 100

    # Training loop
    for epoch in range(num_epochs):
        for x, y in dataloader:
            y = y.view(-1, 1)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()

    return model

model = train_probability_nn(df)

In [37]:
# Check accuracy
def check_accuracy(model, df):
    inputs = df[["M1", "M2", "M3", "M4", "M5"]]
    outputs = df[["Expected"]]
    outputs = outputs.astype(np.float32)

    dataset = torch.utils.data.TensorDataset(torch.tensor(inputs.values, dtype=torch.float32), torch.tensor(outputs.values, dtype=torch.float32))
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=False)

    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for x, y in dataloader:
            y = y.view(-1, 1)
            y_pred = model(x)
            predictions = (y_pred > 0.5).float()  # Convert probabilities to binary predictions
            correct_predictions += (predictions == y).sum().item()
            total_predictions += y.size(0)

    accuracy = correct_predictions / total_predictions * 100
    return accuracy

accuracy = check_accuracy(model, df)

print(f"Final NN accuracy: {accuracy:.2f}%")

Final NN accuracy: 99.24%


In [38]:
# Now add EfficientNetV2 and do the same
# Wyniki dla modelu EfficientNetV2
import torch
from torchvision import transforms, models
from torch import nn, optim
from torch.utils.data import DataLoader
from PIL import Image

model = models.efficientnet_v2_s(weights=None)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)  # Two output neurons

# Load the model weights
path = "/Users/stukeleyak/Desktop/Studia/Doktorat/Projekt/FaceAuthenticityDetection/CrossWalidacja/best_efficientnetv2_model.pth"
model.load_state_dict(torch.load(path))
model.eval()

# Transform the images
# Create a transform to resize images
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Create custom datasets
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label, image_path  # Return image, label, and file path


# Create custom datasets
test_dataset = OurDataset(auth_images + spoof_images, [0] * len(auth_images) + [1] * len(spoof_images), transform=transform)

# Create dataloaders
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model = model.to(device)

# Create a text file to store incorrect predictions
with open("incorrect_predictions_efficientnet.txt", "w") as f:
    f.write("name\tpredicted\tactual\n")
    f.write("(0 - authentic, 1 - spoof)\n")

correct_predictions = 0
total_predictions = 0

# Iterate through the test dataset
for images, labels, image_paths in test_loader:
    images = images.to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = model(images)

    # Get the argmax of each output
    _, predicted = torch.max(outputs, 1)

    total_predictions += labels.size(0)

    correct_predictions += (predicted == labels).sum().item()

    # Write all predictions to the file
    with open("incorrect_predictions_efficientnet.txt", "a") as f:
        for i in range(len(predicted)):
            f.write(f"{image_paths[i]}\t{predicted[i].item()}\t{labels[i].item()}\n")

print("Accuracy for EfficientNetV2 model:")
accuracy = correct_predictions / total_predictions * 100
print(f"Accuracy: {accuracy:.2f}%")

/var/folders/42/03wqtylj4nz_4lpvj_w6mc2h0000gn/T/ipykernel_19957/3850916976.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))


Accuracy for EfficientNetV2 model:
Accuracy: 95.16%


In [39]:
# Prepare data for the final NN
# [M1, M2, M3, M4, M5, M6, Expected]
import pandas as pd
import numpy as np

# Create a dictionary to store the data
data_dict = {}
new_data_dict = {}
second_data_dict = {}

df = pd.DataFrame(columns=["M1", "M2", "M3", "M4", "M5", "M6", "Expected"])

# Read the file and populate the dictionary
with open("ImageResults.txt", "r") as f:
    lines = f.readlines()

    lines = lines[1:]  # Skip the first line

    for line in lines:
        parts = line.strip().split(" ")
        name = parts[0]
        m1 = float(parts[1])
        m2 = float(parts[2])
        m3 = float(parts[3])
        m4 = float(parts[4])

        new_data_dict[name] = [m1, m2, m3, m4]

# Read second file
with open("incorrect_predictions_convnext.txt", "r") as f:
    lines = f.readlines()

    lines = lines[2:]  # Skip the first two lines

    for line in lines:
        parts = line.strip().split("\t")
        name = parts[0]
        name = name.split("/")[-1]  # Get the file name only
        predicted = float(parts[1])
        expected = int(parts[2])

        if name in new_data_dict:
            m1m4 = new_data_dict[name]
            m1 = m1m4[0]
            m2 = m1m4[1]
            m3 = m1m4[2]
            m4 = m1m4[3]

            row = [m1, m2, m3, m4, predicted]

            second_data_dict[name] = row

with open("incorrect_predictions_efficientnet.txt", "r") as f:
    lines = f.readlines()

    lines = lines[2:]  # Skip the first two lines

    for line in lines:
        parts = line.strip().split("\t")
        name = parts[0]
        name = name.split("/")[-1]  # Get the file name only
        predicted = float(parts[1])
        expected = int(parts[2])

        if name in second_data_dict:
            m1m5 = second_data_dict[name]
            m1 = m1m5[0]
            m2 = m1m5[1]
            m3 = m1m5[2]
            m4 = m1m5[3]
            m5 = m1m5[4]

            row = [m1, m2, m3, m4, m5, predicted, expected]

            # Add row to df
            df = pd.concat([df, pd.DataFrame([row], columns=["M1", "M2", "M3", "M4", "M5", "M6", "Expected"])], ignore_index=True)

print("Dataframe length:")
print(len(df))

/var/folders/42/03wqtylj4nz_4lpvj_w6mc2h0000gn/T/ipykernel_19957/2616724625.py:76: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row], columns=["M1", "M2", "M3", "M4", "M5", "M6", "Expected"])], ignore_index=True)


Dataframe length:
22324


In [40]:
# Final NN
# Parameter to control the amount of inputs to the NN
N_INPUTS = 6

class ProbabilityNN(nn.Module):
    def __init__(self):
        super(ProbabilityNN, self).__init__()
        self.fc1 = nn.Linear(N_INPUTS, 10)
        self.fc2 = nn.Linear(10, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [41]:
def train_probability_nn(df):
    inputs = df[["M1", "M2", "M3", "M4", "M5", "M6"]]
    outputs = df[["Expected"]]
    outputs = outputs.astype(np.float32)

    model = ProbabilityNN()

    dataset = torch.utils.data.TensorDataset(torch.tensor(inputs.values, dtype=torch.float32), torch.tensor(outputs.values, dtype=torch.float32))
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 100

    # Training loop
    for epoch in range(num_epochs):
        for x, y in dataloader:
            y = y.view(-1, 1)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()

    return model

model = train_probability_nn(df)

In [43]:
# Check accuracy
def check_accuracy(model, df):
    inputs = df[["M1", "M2", "M3", "M4", "M5", "M6"]]
    outputs = df[["Expected"]]
    outputs = outputs.astype(np.float32)

    dataset = torch.utils.data.TensorDataset(torch.tensor(inputs.values, dtype=torch.float32), torch.tensor(outputs.values, dtype=torch.float32))
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=False)

    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for x, y in dataloader:
            y = y.view(-1, 1)
            y_pred = model(x)
            predictions = (y_pred > 0.5).float()  # Convert probabilities to binary predictions
            correct_predictions += (predictions == y).sum().item()
            total_predictions += y.size(0)

    accuracy = correct_predictions / total_predictions * 100
    return accuracy

accuracy = check_accuracy(model, df)

print(f"Final NN accuracy: {accuracy:.2f}%")

Final NN accuracy: 99.29%


In [45]:
# Now add Vision Transformer and do the same
# Wyniki dla modelu ViT
model = models.vit_b_16(weights=None)   # b_32 for performance, b_16 might be better
num_ftrs = model.heads.head.in_features
model.heads.head = torch.nn.Linear(num_ftrs, 2) # Two output neurons

# Load the model weights
path = "/Users/stukeleyak/Desktop/Studia/Doktorat/Projekt/FaceAuthenticityDetection/CrossWalidacja/best_model_visiontransformer.pth"
model.load_state_dict(torch.load(path))
model.eval()

# Transform the images
# Create a transform to resize images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create custom datasets
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label, image_path  # Return image, label, and file path


# Create custom datasets
test_dataset = OurDataset(auth_images + spoof_images, [0] * len(auth_images) + [1] * len(spoof_images), transform=transform)

# Create dataloaders
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model = model.to(device)

# Create a text file to store incorrect predictions
with open("incorrect_predictions_visiontransformer.txt", "w") as f:
    f.write("name\tpredicted\tactual\n")
    f.write("(0 - authentic, 1 - spoof)\n")

correct_predictions = 0
total_predictions = 0

# Iterate through the test dataset
for images, labels, image_paths in test_loader:
    images = images.to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = model(images)

    # Get the argmax of each output
    _, predicted = torch.max(outputs, 1)

    total_predictions += labels.size(0)

    correct_predictions += (predicted == labels).sum().item()

    # Write all predictions to the file
    with open("incorrect_predictions_visiontransformer.txt", "a") as f:
        for i in range(len(predicted)):
            f.write(f"{image_paths[i]}\t{predicted[i].item()}\t{labels[i].item()}\n")

print("Accuracy for ViT model:")
accuracy = correct_predictions / total_predictions * 100
print(f"Accuracy: {accuracy:.2f}%")

/var/folders/42/03wqtylj4nz_4lpvj_w6mc2h0000gn/T/ipykernel_19957/3974107435.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))


Accuracy for ViT model:
Accuracy: 46.64%


In [46]:
# Prepare data for the final NN
# [M1, M2, M3, M4, M5, M6, Expected]
import pandas as pd
import numpy as np

# Create a dictionary to store the data
data_dict = {}
new_data_dict = {}
second_data_dict = {}
third_data_dict = {}

df = pd.DataFrame(columns=["M1", "M2", "M3", "M4", "M5", "M6", "M7", "Expected"])

# Read the file and populate the dictionary
with open("ImageResults.txt", "r") as f:
    lines = f.readlines()

    lines = lines[1:]  # Skip the first line

    for line in lines:
        parts = line.strip().split(" ")
        name = parts[0]
        m1 = float(parts[1])
        m2 = float(parts[2])
        m3 = float(parts[3])
        m4 = float(parts[4])

        new_data_dict[name] = [m1, m2, m3, m4]

# Read second file
with open("incorrect_predictions_convnext.txt", "r") as f:
    lines = f.readlines()

    lines = lines[2:]  # Skip the first two lines

    for line in lines:
        parts = line.strip().split("\t")
        name = parts[0]
        name = name.split("/")[-1]  # Get the file name only
        predicted = float(parts[1])
        expected = int(parts[2])

        if name in new_data_dict:
            m1m4 = new_data_dict[name]
            m1 = m1m4[0]
            m2 = m1m4[1]
            m3 = m1m4[2]
            m4 = m1m4[3]

            row = [m1, m2, m3, m4, predicted]

            second_data_dict[name] = row

with open("incorrect_predictions_efficientnet.txt", "r") as f:
    lines = f.readlines()

    lines = lines[2:]  # Skip the first two lines

    for line in lines:
        parts = line.strip().split("\t")
        name = parts[0]
        name = name.split("/")[-1]  # Get the file name only
        predicted = float(parts[1])
        expected = int(parts[2])

        if name in second_data_dict:
            m1m5 = second_data_dict[name]
            m1 = m1m5[0]
            m2 = m1m5[1]
            m3 = m1m5[2]
            m4 = m1m5[3]
            m5 = m1m5[4]

            row = [m1, m2, m3, m4, m5, predicted, expected]

            third_data_dict[name] = row

with open("incorrect_predictions_visiontransformer.txt", "r") as f:
    lines = f.readlines()

    lines = lines[2:]  # Skip the first two lines

    for line in lines:
        parts = line.strip().split("\t")
        name = parts[0]
        name = name.split("/")[-1]  # Get the file name only
        predicted = float(parts[1])
        expected = int(parts[2])

        if name in third_data_dict:
            m1m6 = third_data_dict[name]
            m1 = m1m6[0]
            m2 = m1m6[1]
            m3 = m1m6[2]
            m4 = m1m6[3]
            m5 = m1m6[4]
            m6 = m1m6[5]

            row = [m1, m2, m3, m4, m5, m6, predicted, expected]

            # Add row to df
            df = pd.concat([df, pd.DataFrame([row], columns=["M1", "M2", "M3", "M4", "M5", "M6", "M7", "Expected"])], ignore_index=True)

print("Dataframe length:")
print(len(df))

/var/folders/42/03wqtylj4nz_4lpvj_w6mc2h0000gn/T/ipykernel_19957/4166982489.py:102: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row], columns=["M1", "M2", "M3", "M4", "M5", "M6", "M7", "Expected"])], ignore_index=True)


Dataframe length:
22324


In [47]:
# Final NN
# Parameter to control the amount of inputs to the NN
N_INPUTS = 7

class ProbabilityNN(nn.Module):
    def __init__(self):
        super(ProbabilityNN, self).__init__()
        self.fc1 = nn.Linear(N_INPUTS, 10)
        self.fc2 = nn.Linear(10, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [48]:
def train_probability_nn(df):
    inputs = df[["M1", "M2", "M3", "M4", "M5", "M6", "M7"]]
    outputs = df[["Expected"]]
    outputs = outputs.astype(np.float32)

    model = ProbabilityNN()

    dataset = torch.utils.data.TensorDataset(torch.tensor(inputs.values, dtype=torch.float32), torch.tensor(outputs.values, dtype=torch.float32))
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 100

    # Training loop
    for epoch in range(num_epochs):
        for x, y in dataloader:
            y = y.view(-1, 1)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()

    return model

model = train_probability_nn(df)

In [49]:
# Check accuracy
def check_accuracy(model, df):
    inputs = df[["M1", "M2", "M3", "M4", "M5", "M6", "M7"]]
    outputs = df[["Expected"]]
    outputs = outputs.astype(np.float32)

    dataset = torch.utils.data.TensorDataset(torch.tensor(inputs.values, dtype=torch.float32), torch.tensor(outputs.values, dtype=torch.float32))
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=False)

    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for x, y in dataloader:
            y = y.view(-1, 1)
            y_pred = model(x)
            predictions = (y_pred > 0.5).float()  # Convert probabilities to binary predictions
            correct_predictions += (predictions == y).sum().item()
            total_predictions += y.size(0)

    accuracy = correct_predictions / total_predictions * 100
    return accuracy

accuracy = check_accuracy(model, df)

print(f"Final NN accuracy: {accuracy:.2f}%")

Final NN accuracy: 99.34%
